In [1]:
!nvidia-smi

Fri Jun 18 23:58:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks

Mounted at /content/drive
/content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks


In [3]:
# Kaggle API
!pip install --upgrade --force-reinstall --no-deps kaggle
import json
import os
f = open("/content/drive/MyDrive/colab_notebooks/kaggle/kaggle.json", "r")
json_data = json.load(f)
os.environ["KAGGLE_USERNAME"] = json_data["username"]
os.environ["KAGGLE_KEY"] = json_data["key"]

     |████████████████████████████████| 61kB 4.3MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=a98c42f374320959700c5c1ac8661d53a2bc9ba8ae9c4e73e561532817d4e21e
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


This notebook shows how to fine-tune a BERT model (from huggingface) for our dataset recognition task.

Note that internet is needed during the training phase (for downloading the bert-base-cased model). Internet can be turned off during prediction.

In [4]:
# CFG
create_dataset = False

## Install packages

In [5]:
#!pip install datasets --no-index --find-links=file:///kaggle/input/coleridge-packages/packages/datasets
!pip install datasets --no-index --find-links=file:///content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/packages/datasets

Looking in links: file:///content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/packages/datasets
Processing /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/packages/datasets/datasets-1.5.0-py3-none-any.whl
Processing /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/packages/datasets/huggingface_hub-0.0.7-py3-none-any.whl
Processing /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/packages/datasets/xxhash-2.0.0-cp37-cp37m-manylinux2010_x86_64.whl
Processing /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/packages/datasets/fsspec-0.8.7-py3-none-any.whl


In [6]:
!pip install ../input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
!pip install ../input/coleridge-packages/tokenizers-0.10.1-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/coleridge-packages/transformers-4.5.0.dev0-py3-none-any.whl

Processing /content/drive/My Drive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
Processing /content/drive/My Drive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/tokenizers-0.10.1-cp37-cp37m-manylinux1_x86_64.whl
Processing /content/drive/My Drive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/transformers-4.5.0.dev0-py3-none-any.whl
     |████████████████████████████████| 901kB 14.2MB/s 


# Import

In [7]:
import re
import time
import datetime
import random
import glob
import importlib

import numpy as np
import pandas as pd

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import gc

random.seed(123)
np.random.seed(456)

In [8]:
# copy my_seqeval.py to the working directory because the input directory is non-writable
#!cp /kaggle/input/coleridge-packages/my_seqeval.py ./
!cp ../input/coleridge-packages/my_seqeval.py ./

# Hyper-parameters

In [9]:
MAX_LENGTH = 64 #512 #64 # max no. words for each sentence.
OVERLAP = 20 #170 #20 # if a sentence exceeds MAX_LENGTH, we split it to multiple sentences with overlapping

MAX_SAMPLE = None # set a small number for experimentation, set None for production.

# Load data

In [10]:
#papers = {}
#for paper_id in train['Id'].unique():
#    with open(f'{paper_train_folder}/{paper_id}.json', 'r') as f:
#        paper = json.load(f)
#        papers[paper_id] = paper

#!cp -r {paper_train_folder} /content


# Download kagglenb007 output files to local ../input folder
dname = "kagglenb007-get-text"
!mkdir ../input/{dname}
!kaggle kernels output riow1983/{dname} -p ../input/{dname}

mkdir: cannot create directory ‘../input/kagglenb007-get-text’: File exists
Output file downloaded to ../input/kagglenb007-get-text/folds_pubcat.pkl
Kernel log downloaded to ../input/kagglenb007-get-text/kagglenb007-get-text.log 


In [11]:
folds = pd.read_pickle("../input/kagglenb007-get-text/folds_pubcat.pkl")
folds.drop_duplicates(subset=["Id"], keep='first', ignore_index=True, inplace=True)
papers = {k:v for k,v in zip(folds["Id"], folds["text"])}


#train_path = '../input/coleridgeinitiative-show-us-the-data/train.csv'
#paper_train_folder = '../input/coleridgeinitiative-show-us-the-data/train'
#train = pd.read_csv(train_path)
folds = folds[:MAX_SAMPLE]
print(f'No. raw folds rows: {len(folds)}')

# train = train.groupby('Id').agg({
#     'pub_title': 'first',
#     'dataset_title': '|'.join,
#     'dataset_label': '|'.join,
#     'cleaned_label': '|'.join
# }).reset_index()
#print(f'No. grouped training rows: {len(train)}')

No. raw folds rows: 14271


# Transform data to NER format

In [12]:
dname = "nb005-pytorch-bert-for-ner"
#!rm -r {dname}
#!mkdir {dname}

In [13]:
def clean_training_text(txt):
    """
    similar to the default clean_text function but without lowercasing.
    """
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt)).strip()

def shorten_sentences(sentences):
    short_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) > MAX_LENGTH:
            for p in range(0, len(words), MAX_LENGTH - OVERLAP):
                short_sentences.append(' '.join(words[p:p+MAX_LENGTH]))
        else:
            short_sentences.append(sentence)
    return short_sentences

def find_sublist(big_list, small_list):
    all_positions = []
    for i in range(len(big_list) - len(small_list) + 1):
        if small_list == big_list[i:i+len(small_list)]:
            all_positions.append(i)
    
    return all_positions

def tag_sentence(sentence, labels, verbose=False): # requirement: both sentence and labels are already cleaned
    sentence_words = sentence.split()
    
    if labels is not None and any(re.findall(f'\\b{label}\\b', sentence)
                                  for label in labels): # positive sample
        nes = ['O'] * len(sentence_words)
        if verbose:
            print("len(nes): ", len(nes))
        for label in labels:
            label_words = label.split()

            all_pos = find_sublist(sentence_words, label_words)
            for pos in all_pos:
                nes[pos] = 'B'
                for i in range(pos+1, pos+len(label_words)):
                    nes[i] = 'I'

        return True, list(zip(sentence_words, nes))
        
    else: # negative sample
        nes = ['O'] * len(sentence_words)
        return False, list(zip(sentence_words, nes))

In [41]:
#papers['d0fa7568-7d8e-4db9-870f-f9c6f668c17b']

In [14]:
def get_ner_json(df, fold, train_or_valid, verbose=False):
    """
    Args:
        df: pd.DataFrame
        fold: int
        train_or_valid: str
        verbose: bool
    """
    assert train_or_valid in ["train", "valid"], "`train_or_valid` must be either 'train' or 'valid'."
    
    if verbose:
        cnt_pos, cnt_neg = 0, 0 # number of sentences that contain/not contain labels
    
    ner_data = []
    pbar = tqdm(total=len(df))
    for i, id, dataset_label in df[['Id', 'dataset_label']].itertuples():
        # paper
        paper = papers[id]
        
        # labels
        #labels = dataset_label.split('|')
        labels = dataset_label
        labels = [clean_training_text(label) for label in labels]
        
        # sentences
        sentences = set([clean_training_text(sentence) for section in paper 
                    for sentence in section['text'].split('.') 
                    ])
        sentences = shorten_sentences(sentences) # make sentences short
        sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
        
        # positive sample
        for sentence in sentences:
            is_positive, tags = tag_sentence(sentence, labels, verbose=verbose)
            if is_positive:
                if verbose:
                    cnt_pos += 1
                ner_data.append(tags)
            elif any(word in sentence.lower() for word in ['data', 'study']): 
                ner_data.append(tags)
                if verbose:
                    cnt_neg += 1
        
        # process bar
        pbar.update(1)
        if verbose:
            pbar.set_description(f"Training data size: {cnt_pos} positives + {cnt_neg} negatives")

    # shuffling
    random.shuffle(ner_data)


    # write data to file.
    with open(f'./{dname}/fold_{fold}_{train_or_valid}_ner.json', 'w') as f:
        for row in ner_data:
            words, nes = list(zip(*row))
            row_json = {'tokens' : words, 'tags' : nes}
            json.dump(row_json, f)
            f.write('\n')

In [15]:
if create_dataset:
    # get _ner.json files
    for fold in range(5):
        train = folds[folds["fold"]!=fold+1].reset_index(drop=True)
        valid = folds[folds["fold"]==fold+1].reset_index(drop=True)
        get_ner_json(train, fold+1, "train", verbose=False)
        get_ner_json(valid, fold+1, "valid", verbose=False)

# Fine-tune a BERT model for NER

In [ ]:
#os.environ["LOAD_BEST_MODEL_AT_END"] = "true"
#!echo $LOAD_BEST_MODEL_AT_END

In [16]:
#os.environ["OUTPUT_DIR"] = dname

def train(fold=None, resume_training=False, num_checkpoint=None):
    """
    Args:
        fold: int
        resume_training: bool
        num_checkpoint: int
    """
    dname_child = f"fold_{fold}"
    os.environ["OUTPUT_DIR"] = dname+"/"+dname_child
    os.environ["TRAIN_FILE"] = f'./{dname}/fold_{fold}_train_ner.json'
    os.environ["VALID_FILE"] = f'./{dname}/fold_{fold}_valid_ner.json'

    if resume_training:
        if os.path.exists(f"./{dname}/{dname_child}"):
            pass
        else:
            raise KeyError(f"./{dname}/{dname_child} does not exist! Please consider to train w/ resume_training=False.")
        os.environ["MODEL_PATH"] = f"./{dname}/{dname_child}/checkpoint-{num_checkpoint}"
        
        !python ../input/kaggle-ner-utils/kaggle_run_ner.py \
        --model_name_or_path $MODEL_PATH \
        --train_file $TRAIN_FILE \
        --validation_file $VALID_FILE \
        --num_train_epochs 1 \
        --per_device_train_batch_size 8 \
        --per_device_eval_batch_size 8 \
        --save_steps 15000 \
        --output_dir $OUTPUT_DIR \
        --report_to 'none' \
        --seed 123 \
        --do_train \
        --do_eval \
        --evaluation_strategy 'steps' \
        --load_best_model_at_end 'true'

    else:
        if os.path.exists(f"./{dname}/{dname_child}"):
            #!rm -r {dname}/{dname_child}
            raise KeyError(f"./{dname}/{dname_child} exists. Please consider to resume training w/ resume_training=True.")
        else:
            !mkdir {dname}/{dname_child}
        
        !python ../input/kaggle-ner-utils/kaggle_run_ner.py \
        --model_name_or_path 'bert-base-cased' \
        --train_file $TRAIN_FILE \
        --validation_file $VALID_FILE \
        --num_train_epochs 1 \
        --per_device_train_batch_size 8 \
        --per_device_eval_batch_size 8 \
        --save_steps 15000 \
        --output_dir $OUTPUT_DIR \
        --report_to 'none' \
        --seed 123 \
        --do_train \
        --do_eval \
        --evaluation_strategy 'steps' \
        --load_best_model_at_end 'true'

# reference: https://huggingface.co/transformers/main_classes/trainer.html

In [17]:
#  for fold in range(5):
#      print()
#      print(f"#################################### Start training for fold {fold+1} #################################################")
#      train(fold=fold+1, resume_training=False, num_checkpoint=None)

train(fold=3, resume_training=False, num_checkpoint=None)
#train(resume_training=True, num_checkpoint=45000)

Streaming output truncated to the last 5000 lines.
 59% 1557/2635 [00:44<00:29, 37.08it/s]
 59% 1561/2635 [00:44<00:31, 34.22it/s]
 59% 1565/2635 [00:44<00:30, 34.76it/s]
 60% 1570/2635 [00:44<00:29, 36.44it/s]
 60% 1574/2635 [00:44<00:28, 37.40it/s]
 60% 1578/2635 [00:45<00:28, 36.82it/s]
 60% 1582/2635 [00:45<00:29, 35.75it/s]
 60% 1586/2635 [00:45<00:31, 33.82it/s]
 60% 1590/2635 [00:45<00:31, 33.56it/s]
 60% 1594/2635 [00:45<00:31, 32.82it/s]
 61% 1598/2635 [00:45<00:31, 32.47it/s]
 61% 1602/2635 [00:45<00:30, 33.92it/s]
 61% 1606/2635 [00:45<00:29, 34.55it/s]
 61% 1611/2635 [00:46<00:28, 35.87it/s]
 61% 1615/2635 [00:46<00:28, 36.40it/s]
 61% 1619/2635 [00:46<00:28, 36.06it/s]
 62% 1623/2635 [00:46<00:30, 33.14it/s]
 62% 1627/2635 [00:46<00:30, 33.22it/s]
 62% 1631/2635 [00:46<00:29, 33.71it/s]
 62% 1635/2635 [00:46<00:28, 34.65it/s]
 62% 1639/2635 [00:46<00:28, 35.17it/s]
 62% 1643/2635 [00:46<00:28, 34.42it/s]
 63% 1647/2635 [00:47<00:27, 35.37it/s]
 63% 1652/2635 [00:47<00:26, 

In [19]:
train(fold=2, resume_training=False, num_checkpoint=None)

KeyError: ignored

After the tuning finishes, we should find our model in './nb005-pytorch-bert-for-ner'.

# Upload to Kaggle
./nb005-pytorch-bert-for-nerをKaggle datasetsにupload

In [5]:
dname = "nb005-pytorch-bert-for-ner"
# [Init] -----------------------------------------------------------------------------------
# #!mkdir {dname}
# !kaggle datasets init -p {dname}
# # referene: https://kaeru-nantoka.hatenablog.com/entry/2020/01/17/015551

# with open(f"{dname}/dataset-metadata.json", "r") as jsonFile:
#     data = json.load(jsonFile)

# data["id"] = f"riow1983/{dname}"
# data["title"] = dname

# with open(f"{dname}/dataset-metadata.json", "w") as jsonFile:
#     json.dump(data, jsonFile)

# # Copy train_ner.json -> replaced w/ location specification during saving time
# #!cp train_ner.json ./{dname}/train_ner.json

# !kaggle datasets create -p {dname}




# [Update] -----------------------------------------------------------------------------------
!kaggle datasets version -p {dname} -m "[Update] model.bin file added"

Starting upload for file fold_1_train_ner.json
100% 75.5M/75.5M [00:04<00:00, 19.5MB/s]
Upload successful: fold_1_train_ner.json (75MB)
Starting upload for file fold_1_valid_ner.json
100% 141M/141M [00:04<00:00, 30.0MB/s]
Upload successful: fold_1_valid_ner.json (141MB)
Starting upload for file fold_2_train_ner.json
100% 151M/151M [00:05<00:00, 27.7MB/s]
Upload successful: fold_2_train_ner.json (151MB)
Starting upload for file fold_2_valid_ner.json
100% 65.8M/65.8M [00:03<00:00, 18.4MB/s]
Upload successful: fold_2_valid_ner.json (66MB)
Starting upload for file fold_3_train_ner.json
100% 208M/208M [00:06<00:00, 34.1MB/s]
Upload successful: fold_3_train_ner.json (208MB)
Starting upload for file fold_3_valid_ner.json
100% 8.09M/8.09M [00:04<00:00, 1.98MB/s]
Upload successful: fold_3_valid_ner.json (8MB)
Starting upload for file fold_4_train_ner.json
100% 216M/216M [00:05<00:00, 43.6MB/s]
Upload successful: fold_4_train_ner.json (216MB)
Starting upload for file fold_4_valid_ner.json
100% 8

In [ ]:
# Copy train_ner.json
#!cp train_ner.json ./{dname}/train_ner.json
#!kaggle datasets version -p {dname} -m "[Add] train_ner.json"

Skipping folder: checkpoint-15000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-30000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-45000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-60000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-75000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-90000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-105000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-120000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-135000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-150000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-165000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-180000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-195000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-210000; use '--dir-mode' to upload folders
Skipping fol